## Overview
Is the recommended way to build images. Dockerfile uses a simple domain specific languae containing instructions to builda Doker image. The other method of building images is through `docker commit` command. A very simple Dockerfile would look like:

```
FROM ubuntu:22.04
COPY . /app
RUN make /app
CMD python /app/app.py
```

Which essentially means - use Ubuntu as the base image, copy the current directory to /app directory within the container. Execute the make command. Finally, run the python command.

Some characteristics of the Dockerfile:
- Every line has the format `INSTRUCTION arguments`. 
- A Dockerfile must begin with a `FROM` instruction. 
- Docker treats lines that begin with `#` as a comment.
- Leading whitespace before comments `#` and instructions (such as `RUN`) are ignored, but discouraged.

## Build Image
To build image from a Dockerfile, we use `docker build [OPTIONS] PATH | URL | -` command. With the`-t` option, we can name and tag the image. Consider the simple Dockerfile:
```
FROM alpine
RUN apk add curl
```

```bash
$ docker build -t="example/curl_image" .
[+] Building 9.0s (6/6) FINISHED                                                               docker:default
 => [internal] load .dockerignore                                                                        0.0s
 => => transferring context: 2B                                                                          0.0s
 => [internal] load build definition from Dockerfile                                                     0.0s
 => => transferring dockerfile: 66B                                                                      0.0s
 => [internal] load metadata for docker.io/library/alpine:latest                                         0.0s
 => CACHED [1/2] FROM docker.io/library/alpine                                                           0.0s
 => [2/2] RUN apk add curl                                                                               8.6s
 => exporting to image                                                                                   0.1s
 => => exporting layers                                                                                  0.1s
 => => writing image sha256:d04d900f6edd26e4db89974d85f2f36a7f8c709b3238c94d10df33f32c0f90a0             0.0s
 => => naming to docker.io/example/curl_image
```
Trailing dot in the example above instructs Docker to look for Dockerfile in the current directory.

## Instruction Reference
### `FROM`
The syntax looks like: `FROM [--platform=<platform>] <image>[:<tag>] [AS <name>]`. As specified earlier, `FROM` must be te first instruction - excxept for `ARG` instruction. Example:

```
ARG VERSION=latest
FROM busybox:$VERSION
```

### `RUN`
The `RUN` command has two forms:
- `RUN <command>` (shell form)
- `RUN ["executable", "param1", "param2"]` (exec form)

Unlike the shell form, the exec form does not invoke a command shell. This means that normal shell processing does not happen. For example, `RUN [ "echo", "$HOME" ]` will not do variable substitution on `$HOME`. If we want shell processing then either use the shell form or execute a shell directly, for example: `RUN [ "sh", "-c", "echo $HOME" ]`.

```
FROM ubuntu:18.04
RUN apt-get update; apt-get install -y nginx
```

### `CMD`
`CMD` command like the previous command has multiple forms:
- `CMD ["executable","param1","param2"]` (exec form, this is the preferred form)
- `CMD ["param1","param2"]` (as default parameters to `ENTRYPOINT`)
- `CMD command param1 param2` (shell form)

This command specifies a command to run when a container is launched. It is similar to `RUN` instruction but rather than running the command when the container is being built, it will specify the command to run when launching the container.

It is important to understand that we can override the `CMD` instruction using `docker run` command. For example if we ran `docker run -t -i example/example_image`, it will use the command to be exexuted from the Dockerfile `CMD` instruction. However, if we ran `docker run -t -i example/example_image /bin/ps` it will override the Dockerfile `CMD` instruction.

If more than one `CMD` instructions are present in the Dockerfile, the last one will be used.

### `ENTRYPOINT`
Has two forms similar to what we have with `RUN` - shell and exec form. `ENTRYPOINT` provides a command that isn;t easily overridden. Instead any argument passed to the `docker run` commandline will be passed as argument to command specified in `ENTRYPOINT`.

```
ENTRYPOINT ["/usr/sbin/nginx"]
CMD ["-h]
```

```bash
$ docker run example/nginx_img
```
Running the above executes `/usr/sbin/nginx -h` command. However, if we run:
```bash
$ docker run example/nginx_img -g "daemon_off"
```
it executes `/usr/sbin/nginx -g "daemon_off"`

### `ENV`
Sets environment variable during build process. These environment variables will persist into any container built from this image.

```
ENV JAVA_HOME /usr/lib/jvm/
# Use the previously defined environment variable
WORKDIR $JAVA_HOME
```

### `ADD`
The `ADD` instruction copies new files, directories or remote file URLs from `<src>` and adds them to the filesystem of the image at the path `<dest>`. The full syntax looks like: `ADD [--chown=<user>:<group>] [--chmod=<perms>] [--checksum=<checksum>] <src>... <dest>`. Source path may also contain wildcards. Destination path can be absolute or relative to `WORKDIR`. If the destination ends in a /, then Docker considers it as directory, else it considers it as file.

```
# Adds all markdown files in build environment into /var/docs directory
ADD *.md /var/docs/
```

There are some additional features provided by `ADD`. Docker will unpack an added tar file.

### `COPY`
Same as `ADD` except for lack of extraction or decompression capabilities. 

```
# Copy files from conf.d directory to /etc/apache2 directory
COPY conf.d/ /etc/apache2
```

### `WORKDIR`
The `WORKDIR` instruction sets the working directory for any `RUN`, `CMD`, `ENTRYPOINT`, `COPY` and `ADD` instructions that follow it in the Dockerfile. If the `WORKDIR` doesn't exist, it will be created even if it's not used in any subsequent Dockerfile instruction.

### `EXPOSE`
Informs Docker that the container listens to the given port at runtime. Syntax: `EXPOSE <port> [<port>/<protocol>...]`.

It is important to note that exposing a port doesn't mean that we can automatically access the service running on that port, it is merely a documentation between the person who builds the image and the person who runs the container. Docker would open a port and bind it to a random local port when we use the `-p` option while using `docker run`.

```
EXPOSE 80
```
```bash
$ docker run -d -p 80 example/simple_server
```
This will open a random port on Docker host that will connect to port 80 on the container. To specify a port number to use:
```bash
$ docker run -d -p 8080:80 example/simple_server
```
Using `-P` option assigns random host ports to all container exposed ports.

### `VOLUME`
Creates a mount point with the specified name and marks it as holding externally mounted volumes from native host or other containers. The value can be an array of a simple string

```
VOLUME ["/var/log/"]
# Or
VOLUME /var/log
# Even multiple values
VOLUME /var/log /var/config
```